# Replicating Nielson's Handwritten Digit Recognition Neural Net

## Data & Decisions In-Class Assignment (Nov 20)

Reference: Neural Networks & Deep Learning (Michael Nielson)
[Chapter 1: Using neural nets to recognize handwritten digits](https://neuralnetworksanddeeplearning.com/chap1.html)

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

In [10]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a): # Add a feedforward method to the Network class, which, given an input a for the network, returns the corresponding output
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):  # Ofc the main thing we want our Network objects to do is to learn. To that end give them an SGD method which implements stochastic gradient descent
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs): # In each epoch, it starts by randomly shuffling the training data, and then partitions it into mini-batches of the appropriate size
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches: # This is an easy way of sampling randomly from the training data. Then for each mini_batch we apply a single step of gradient descent
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [11]:
# Instead of using the mnist_loader.py file as in Nielson's example, I'm using TensorFlow's built-in MNIST dataset loader

def load_data_wrapper():
    (train_X, train_y), (test_X, test_y) = mnist.load_data()

    # Reshape and normalize the data
    train_X = train_X.reshape(train_X.shape[0], 784, 1) / 255.0
    test_X = test_X.reshape(test_X.shape[0], 784, 1) / 255.0

    # Convert labels to one-hot encoded vectors
    def vectorized_result(j):
        e = np.zeros((10, 1))
        e[j] = 1.0
        return e

    train_y = [vectorized_result(y) for y in train_y]

    # Split test data into validation and test sets
    validation_data = list(zip(test_X[:5000], test_y[:5000]))
    test_data = list(zip(test_X[5000:], test_y[5000:]))

    # Prepare training data
    training_data = list(zip(train_X, train_y))

    return (training_data, validation_data, test_data)

# Load the data
training_data, validation_data, test_data = load_data_wrapper()

print("Data loaded successfully!")
print(f"Training data size: {len(training_data)}")
print(f"Validation data size: {len(validation_data)}")
print(f"Test data size: {len(test_data)}")

Data loaded successfully!
Training data size: 60000
Validation data size: 5000
Test data size: 5000


In [12]:
# Instantiate the network
net = Network([784, 30, 10]) # creates a network with 784 input neurons (28x28 pixel images), 30 hidden neurons, 10 output neurons (for the 10 digit classes)

In [13]:
# Use stochastic gradient descent to learn from the MNIST data over 30 epochs with a mini-batch size of 10 and learning rate η = 3.0 (these values are provided by Nielson)
net.SGD(training_data, 30, 10, 3.0, test_data=test_data) # The transcript shows the number of test images correctly recognized by the neural network after each epoch

Epoch 0: 4648 / 5000
Epoch 1: 4712 / 5000
Epoch 2: 4751 / 5000
Epoch 3: 4761 / 5000
Epoch 4: 4777 / 5000
Epoch 5: 4782 / 5000
Epoch 6: 4783 / 5000
Epoch 7: 4805 / 5000
Epoch 8: 4782 / 5000
Epoch 9: 4799 / 5000
Epoch 10: 4790 / 5000
Epoch 11: 4813 / 5000
Epoch 12: 4823 / 5000
Epoch 13: 4832 / 5000
Epoch 14: 4815 / 5000
Epoch 15: 4817 / 5000
Epoch 16: 4811 / 5000
Epoch 17: 4812 / 5000
Epoch 18: 4818 / 5000
Epoch 19: 4833 / 5000
Epoch 20: 4820 / 5000
Epoch 21: 4824 / 5000
Epoch 22: 4821 / 5000
Epoch 23: 4824 / 5000
Epoch 24: 4817 / 5000
Epoch 25: 4828 / 5000
Epoch 26: 4836 / 5000
Epoch 27: 4846 / 5000
Epoch 28: 4837 / 5000
Epoch 29: 4831 / 5000


Ok so it looks like generally, the number of correctly recognized images increases over the epochs. And even after just a single epoch this has reached 4648 out of 5000 -- which is pretty significant (92.96%). Over all the epochs, the highest correst classifcation rate is for Epoch 27 which has 96.92%.

In [15]:
# Rerunning the above experiment, changing the number of hidden neurons to 100
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 3247 / 5000
Epoch 1: 3745 / 5000
Epoch 2: 3776 / 5000
Epoch 3: 3791 / 5000
Epoch 4: 3795 / 5000
Epoch 5: 3827 / 5000
Epoch 6: 3827 / 5000
Epoch 7: 4357 / 5000
Epoch 8: 4365 / 5000
Epoch 9: 4382 / 5000
Epoch 10: 4380 / 5000
Epoch 11: 4389 / 5000
Epoch 12: 4374 / 5000
Epoch 13: 4390 / 5000
Epoch 14: 4393 / 5000
Epoch 15: 4389 / 5000
Epoch 16: 4390 / 5000
Epoch 17: 4392 / 5000
Epoch 18: 4399 / 5000
Epoch 19: 4399 / 5000
Epoch 20: 4390 / 5000
Epoch 21: 4395 / 5000
Epoch 22: 4398 / 5000
Epoch 23: 4399 / 5000
Epoch 24: 4396 / 5000
Epoch 25: 4398 / 5000
Epoch 26: 4400 / 5000
Epoch 27: 4395 / 5000
Epoch 28: 4400 / 5000
Epoch 29: 4403 / 5000


Ok it looks like when we increase the no. of neurons from 30 to 100, our results are worse. The best classification rate is 4403/5000 or 88.06% in the last Epoch 29. In Nielson's example, increasing the no. of neurons gave him better results, but he does have a comment noting that reader feedback indicates a lot of variation for this experiment.

At this point, Nielson points out that to obtain such accuracies he had to make specific choices for the # epochs of training, mini-batch size, and η (hyper-parameters). So in the next step, I choose a different set of hyper-parameters, setting η to 0.001 (as suggested):

In [17]:
net = Network([784, 100, 10])
net.SGD(training_data, 30, 10, 0.001, test_data=test_data)

Epoch 0: 478 / 5000
Epoch 1: 497 / 5000
Epoch 2: 496 / 5000
Epoch 3: 496 / 5000
Epoch 4: 496 / 5000
Epoch 5: 496 / 5000
Epoch 6: 497 / 5000
Epoch 7: 497 / 5000
Epoch 8: 497 / 5000
Epoch 9: 497 / 5000
Epoch 10: 496 / 5000
Epoch 11: 497 / 5000
Epoch 12: 497 / 5000
Epoch 13: 497 / 5000
Epoch 14: 497 / 5000
Epoch 15: 497 / 5000
Epoch 16: 497 / 5000
Epoch 17: 498 / 5000
Epoch 18: 498 / 5000
Epoch 19: 501 / 5000
Epoch 20: 501 / 5000
Epoch 21: 502 / 5000
Epoch 22: 503 / 5000
Epoch 23: 507 / 5000
Epoch 24: 508 / 5000
Epoch 25: 509 / 5000
Epoch 26: 515 / 5000
Epoch 27: 522 / 5000
Epoch 28: 523 / 5000
Epoch 29: 527 / 5000


Anddd it looks like the results are pretty bad. Best classification rate is 527/5000 which is 10.54% -- which is big drop from our previous rate of almost 97%. However, as Nielson points out, we can definitely see that performance is getting better over epochs, which suggests that we must increase the learning rate.

We can try increasing the learning rate to 100, trying it out with 30 neurons:

In [18]:
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 100.0, test_data=test_data)

Epoch 0: 502 / 5000
Epoch 1: 761 / 5000
Epoch 2: 762 / 5000
Epoch 3: 763 / 5000
Epoch 4: 765 / 5000
Epoch 5: 767 / 5000
Epoch 6: 768 / 5000
Epoch 7: 768 / 5000
Epoch 8: 768 / 5000
Epoch 9: 770 / 5000
Epoch 10: 769 / 5000
Epoch 11: 772 / 5000
Epoch 12: 776 / 5000
Epoch 13: 772 / 5000
Epoch 14: 775 / 5000
Epoch 15: 776 / 5000
Epoch 16: 779 / 5000
Epoch 17: 783 / 5000
Epoch 18: 775 / 5000
Epoch 19: 771 / 5000
Epoch 20: 766 / 5000
Epoch 21: 760 / 5000
Epoch 22: 759 / 5000
Epoch 23: 756 / 5000
Epoch 24: 750 / 5000
Epoch 25: 747 / 5000
Epoch 26: 746 / 5000
Epoch 27: 745 / 5000
Epoch 28: 743 / 5000
Epoch 29: 744 / 5000


So evidently, the results are still pretty bad, the learning rate is too high. The best classification is in Epoch 17 (783/5000) 15.66%, which is slightly better than the previous case where we used a 100 neurons and a learning rate of 0.01. However, here it does look like performance isn't getting better over the epochs? At least not consistently increasing, because it drops towards the later epochs.



---



### **Exercise**: Try creating a network with just two layers - an input and an output layer, no hidden layer - with 784 and 10 neurons, respectively. Train the network using stochastic gradient descent. What classification accuracy can you achieve?

#### In the below cells, I try out a bunch of different combinations of hyperparameters.


### Trial 1
* no. epochs = 30
* mini-batch size = 10
* eta = 3.0

In [20]:
net = Network([784, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 1980 / 5000
Epoch 1: 2003 / 5000
Epoch 2: 2552 / 5000
Epoch 3: 3044 / 5000
Epoch 4: 3413 / 5000
Epoch 5: 3409 / 5000
Epoch 6: 3412 / 5000
Epoch 7: 3418 / 5000
Epoch 8: 3419 / 5000
Epoch 9: 3407 / 5000
Epoch 10: 3409 / 5000
Epoch 11: 3416 / 5000
Epoch 12: 3413 / 5000
Epoch 13: 3414 / 5000
Epoch 14: 3413 / 5000
Epoch 15: 3417 / 5000
Epoch 16: 3420 / 5000
Epoch 17: 3418 / 5000
Epoch 18: 3416 / 5000
Epoch 19: 3413 / 5000
Epoch 20: 3415 / 5000
Epoch 21: 3420 / 5000
Epoch 22: 3424 / 5000
Epoch 23: 3418 / 5000
Epoch 24: 3419 / 5000
Epoch 25: 3420 / 5000
Epoch 26: 3420 / 5000
Epoch 27: 3419 / 5000
Epoch 28: 3420 / 5000
Epoch 29: 3421 / 5000


### Trial 2
* no. epochs = 30
* mini-batch size = 10
* eta = 0.1

In [21]:
net = Network([784, 10])
net.SGD(training_data, 30, 10, 0.1, test_data=test_data)

Epoch 0: 1856 / 5000
Epoch 1: 2196 / 5000
Epoch 2: 2455 / 5000
Epoch 3: 2572 / 5000
Epoch 4: 2611 / 5000
Epoch 5: 2649 / 5000
Epoch 6: 2682 / 5000
Epoch 7: 2704 / 5000
Epoch 8: 2715 / 5000
Epoch 9: 2749 / 5000
Epoch 10: 2771 / 5000
Epoch 11: 2821 / 5000
Epoch 12: 2894 / 5000
Epoch 13: 2996 / 5000
Epoch 14: 3094 / 5000
Epoch 15: 3127 / 5000
Epoch 16: 3141 / 5000
Epoch 17: 3159 / 5000
Epoch 18: 3167 / 5000
Epoch 19: 3172 / 5000
Epoch 20: 3178 / 5000
Epoch 21: 3177 / 5000
Epoch 22: 3182 / 5000
Epoch 23: 3185 / 5000
Epoch 24: 3193 / 5000
Epoch 25: 3192 / 5000
Epoch 26: 3192 / 5000
Epoch 27: 3199 / 5000
Epoch 28: 3208 / 5000
Epoch 29: 3201 / 5000


### Trial 3
* no. epochs = 30
* mini-batch size = 10
* eta = 6

In [22]:
net = Network([784, 10])
net.SGD(training_data, 30, 10, 6, test_data=test_data)

Epoch 0: 3770 / 5000
Epoch 1: 3807 / 5000
Epoch 2: 3804 / 5000
Epoch 3: 3814 / 5000
Epoch 4: 3837 / 5000
Epoch 5: 3812 / 5000
Epoch 6: 3832 / 5000
Epoch 7: 3816 / 5000
Epoch 8: 3839 / 5000
Epoch 9: 3834 / 5000
Epoch 10: 3865 / 5000
Epoch 11: 4287 / 5000
Epoch 12: 4274 / 5000
Epoch 13: 4288 / 5000
Epoch 14: 4290 / 5000
Epoch 15: 4273 / 5000
Epoch 16: 4286 / 5000
Epoch 17: 4288 / 5000
Epoch 18: 4302 / 5000
Epoch 19: 4282 / 5000
Epoch 20: 4300 / 5000
Epoch 21: 4283 / 5000
Epoch 22: 4316 / 5000
Epoch 23: 4303 / 5000
Epoch 24: 4302 / 5000
Epoch 25: 4294 / 5000
Epoch 26: 4312 / 5000
Epoch 27: 4282 / 5000
Epoch 28: 4307 / 5000
Epoch 29: 4288 / 5000


### Trial 4
* no. epochs = 30
* mini-batch size = 10
* eta = 12

In [24]:
net = Network([784, 10])
net.SGD(training_data, 30, 10, 12, test_data=test_data)

Epoch 0: 3839 / 5000
Epoch 1: 3878 / 5000
Epoch 2: 4236 / 5000
Epoch 3: 4338 / 5000
Epoch 4: 4367 / 5000
Epoch 5: 4374 / 5000
Epoch 6: 4390 / 5000
Epoch 7: 4424 / 5000
Epoch 8: 4434 / 5000
Epoch 9: 4423 / 5000
Epoch 10: 4415 / 5000
Epoch 11: 4447 / 5000
Epoch 12: 4491 / 5000
Epoch 13: 4488 / 5000
Epoch 14: 4521 / 5000
Epoch 15: 4537 / 5000
Epoch 16: 4515 / 5000
Epoch 17: 4522 / 5000
Epoch 18: 4515 / 5000
Epoch 19: 4502 / 5000
Epoch 20: 4490 / 5000
Epoch 21: 4521 / 5000
Epoch 22: 4525 / 5000
Epoch 23: 4471 / 5000
Epoch 24: 4543 / 5000
Epoch 25: 4533 / 5000
Epoch 26: 4523 / 5000
Epoch 27: 4534 / 5000
Epoch 28: 4550 / 5000
Epoch 29: 4557 / 5000


### Trial 5
* no. epochs = 50
* mini-batch size = 10
* eta = 5

In [25]:
net = Network([784, 10])
net.SGD(training_data, 50, 10, 5.0, test_data=test_data)

Epoch 0: 3852 / 5000
Epoch 1: 3872 / 5000
Epoch 2: 4241 / 5000
Epoch 3: 4284 / 5000
Epoch 4: 4275 / 5000
Epoch 5: 4294 / 5000
Epoch 6: 4293 / 5000
Epoch 7: 4266 / 5000
Epoch 8: 4294 / 5000
Epoch 9: 4296 / 5000
Epoch 10: 4296 / 5000
Epoch 11: 4287 / 5000
Epoch 12: 4293 / 5000
Epoch 13: 4302 / 5000
Epoch 14: 4322 / 5000
Epoch 15: 4295 / 5000
Epoch 16: 4316 / 5000
Epoch 17: 4310 / 5000
Epoch 18: 4307 / 5000
Epoch 19: 4312 / 5000
Epoch 20: 4326 / 5000
Epoch 21: 4324 / 5000
Epoch 22: 4329 / 5000
Epoch 23: 4327 / 5000
Epoch 24: 4323 / 5000
Epoch 25: 4330 / 5000
Epoch 26: 4332 / 5000
Epoch 27: 4319 / 5000
Epoch 28: 4312 / 5000
Epoch 29: 4317 / 5000
Epoch 30: 4347 / 5000
Epoch 31: 4335 / 5000
Epoch 32: 4333 / 5000
Epoch 33: 4322 / 5000
Epoch 34: 4322 / 5000
Epoch 35: 4339 / 5000
Epoch 36: 4345 / 5000
Epoch 37: 4330 / 5000
Epoch 38: 4350 / 5000
Epoch 39: 4337 / 5000
Epoch 40: 4335 / 5000
Epoch 41: 4354 / 5000
Epoch 42: 4337 / 5000
Epoch 43: 4350 / 5000
Epoch 44: 4338 / 5000
Epoch 45: 4352 / 500

### Trial 6
* no. epochs = 50
* mini-batch size = 10
* eta = 15.0

In [26]:
net = Network([784, 10])
net.SGD(training_data, 50, 10, 15.0, test_data=test_data)

Epoch 0: 1888 / 5000
Epoch 1: 3004 / 5000
Epoch 2: 3151 / 5000
Epoch 3: 3181 / 5000
Epoch 4: 3573 / 5000
Epoch 5: 3610 / 5000
Epoch 6: 3587 / 5000
Epoch 7: 3652 / 5000
Epoch 8: 3629 / 5000
Epoch 9: 3670 / 5000
Epoch 10: 3635 / 5000
Epoch 11: 3667 / 5000
Epoch 12: 3658 / 5000
Epoch 13: 3688 / 5000
Epoch 14: 3701 / 5000
Epoch 15: 3697 / 5000
Epoch 16: 3796 / 5000
Epoch 17: 3900 / 5000
Epoch 18: 3909 / 5000
Epoch 19: 3947 / 5000
Epoch 20: 3891 / 5000
Epoch 21: 3930 / 5000
Epoch 22: 3951 / 5000
Epoch 23: 3966 / 5000


<ipython-input-10-147c46daefdc>:110: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


Epoch 24: 3940 / 5000
Epoch 25: 3933 / 5000
Epoch 26: 3969 / 5000
Epoch 27: 3948 / 5000
Epoch 28: 3967 / 5000
Epoch 29: 3972 / 5000
Epoch 30: 3955 / 5000
Epoch 31: 3974 / 5000
Epoch 32: 3981 / 5000
Epoch 33: 3967 / 5000
Epoch 34: 3992 / 5000
Epoch 35: 3993 / 5000
Epoch 36: 3956 / 5000
Epoch 37: 4012 / 5000
Epoch 38: 4011 / 5000
Epoch 39: 3995 / 5000
Epoch 40: 3990 / 5000
Epoch 41: 4016 / 5000
Epoch 42: 3994 / 5000
Epoch 43: 3968 / 5000
Epoch 44: 3994 / 5000
Epoch 45: 4002 / 5000
Epoch 46: 3981 / 5000
Epoch 47: 4004 / 5000
Epoch 48: 3971 / 5000
Epoch 49: 3981 / 5000


### Trial 7
* no. epochs = 40
* mini-batch size = 10
* eta = 12.0


In [27]:
net = Network([784, 10])
net.SGD(training_data, 40, 10, 12.0, test_data=test_data)


Epoch 0: 3419 / 5000
Epoch 1: 3496 / 5000
Epoch 2: 3942 / 5000
Epoch 3: 4027 / 5000
Epoch 4: 3989 / 5000
Epoch 5: 4026 / 5000
Epoch 6: 4096 / 5000
Epoch 7: 4089 / 5000
Epoch 8: 4117 / 5000
Epoch 9: 4092 / 5000
Epoch 10: 4119 / 5000
Epoch 11: 4082 / 5000
Epoch 12: 4278 / 5000
Epoch 13: 4387 / 5000
Epoch 14: 4445 / 5000
Epoch 15: 4454 / 5000
Epoch 16: 4372 / 5000
Epoch 17: 4504 / 5000
Epoch 18: 4514 / 5000
Epoch 19: 4492 / 5000
Epoch 20: 4489 / 5000
Epoch 21: 4442 / 5000
Epoch 22: 4482 / 5000
Epoch 23: 4564 / 5000
Epoch 24: 4509 / 5000
Epoch 25: 4512 / 5000
Epoch 26: 4508 / 5000
Epoch 27: 4453 / 5000
Epoch 28: 4541 / 5000
Epoch 29: 4545 / 5000
Epoch 30: 4546 / 5000
Epoch 31: 4499 / 5000
Epoch 32: 4547 / 5000
Epoch 33: 4572 / 5000
Epoch 34: 4550 / 5000
Epoch 35: 4530 / 5000
Epoch 36: 4577 / 5000
Epoch 37: 4561 / 5000
Epoch 38: 4525 / 5000
Epoch 39: 4546 / 5000


### Looks like trial 4 with the following hyperparameters performed best.
* no. epochs = 30
* mini-batch size = 10
* eta = 12


Which has a correct classification rate of 4557/5000 test images (91.14%). I haven't been able to improve it further, and I think it could be because the two layer archictecture is insufficient.


---

